In [1]:
from keras import Input, layers

In [2]:
input_tensor = Input(shape=(32,))

In [3]:
#層は関数
#出力空間次元数=32
dense = layers.Dense(32, activation='relu' )

In [4]:
output_tensor = dense(input_tensor)

In [5]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

In [6]:
#Sequentialモデル
seq_model = Sequential()

In [7]:
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

In [8]:
#FunctionalAPI
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

In [9]:
#Model は入力テンソルと出力テンソルをモデルに変換する
model = Model(input_tensor, output_tensor)

In [10]:
#modelのアーキ確認
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [11]:
#モデルコンパイル
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [12]:
#ダミーデータ
import numpy as np

In [13]:
#連続一様分布で[0,1)からランダム生成
x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

In [14]:
#10エポックで訓練
model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 11.6924
Epoch 2/10
1000/1000 [==============================] - 0s 29us/step - loss: 11.6525
Epoch 3/10
1000/1000 [==============================] - 0s 29us/step - loss: 11.6439
Epoch 4/10
1000/1000 [==============================] - 0s 29us/step - loss: 11.6402
Epoch 5/10
1000/1000 [==============================] - 0s 27us/step - loss: 11.6376
Epoch 6/10
1000/1000 [==============================] - 0s 29us/step - loss: 11.6357
Epoch 7/10
1000/1000 [==============================] - 0s 27us/step - loss: 11.6337
Epoch 8/10
1000/1000 [==============================] - 0s 28us/step - loss: 11.6326
Epoch 9/10
1000/1000 [==============================] - 0s 26us/step - loss: 11.6309
Epoch 10/10
1000/1000 [==============================] - 0s 28us/step - loss: 11.6297


In [15]:
#評価
score = model.evaluate(x_train, y_train)

1000/1000 [==============================] - 0s 70us/step


<h3>2つの入力を持つ場合

In [16]:
from keras.models import Model
from keras import layers
from keras import Input

In [17]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
anser_vocabulary_size = 500

In [18]:
#テキスト入力は整数の可変長シーケンス
#必要であれば、入力に名前をつけられる
text_input = Input(shape=(None,), dtype='int32', name='text')

In [20]:
#入力をサイズ64のベクトルシーケンスに埋め込む
# embedded_text = layers.Embedding(text_vocabulary_size)(text_input, 64)
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)

In [21]:
#LSTMでこれらのベクトルを単一のベクトルにエンコード
#出力の次元数=32
encoded_text = layers.LSTM(32)(embedded_text)

In [22]:
#質問入力でも同じプロセスを繰り返す
question_input = Input(shape=(None,), dtype='int32', name='question')

In [23]:
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)

In [24]:
encoded_question = layers.LSTM(16)(embedded_question)

In [25]:
#テキストと質問を連結
#最後の次元で連結
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

In [26]:
#ソフトマックス分類器
answer = layers.Dense(anser_vocabulary_size, activation='softmax')(concatenated)

In [27]:
#modelインスタンス化
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics =['acc'])

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LS

<h3>多入力モデルへのデータ供給

In [29]:
import numpy as np

In [30]:
num_samples = 1000
max_length = 100

In [31]:
#ダミーデータ生成
#1~9999のデータ、(1000,100)の形状
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))

In [32]:
text

array([[7343, 6712, 5077, ..., 9475, 5336,   35],
       [4996,  514, 9277, ..., 1851, 9701, 4892],
       [1398, 2619, 3024, ..., 7566, 6287, 8210],
       ...,
       [8159, 2153, 4721, ..., 7087,  484,    5],
       [4343, 1269,  193, ..., 1629, 1526, 6903],
       [8249,  638, 3559, ...,  992, 8122, 7160]])

In [33]:
text.shape

(1000, 100)

In [34]:
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

In [35]:
#答えにone-hotエンコーディング適用,(1000,500)
answers = np.zeros(shape=(num_samples, anser_vocabulary_size))

In [36]:
#0~499までの整数,(1000,1)の形状のデータ
indices = np.random.randint(0, anser_vocabulary_size, size=num_samples)

In [37]:
for i, x in enumerate(answers):
    x[indices[i]] = 1

In [38]:
#入力リストを使った適合
model.fit([text,question], answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.2146 - acc: 0.0010
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1995 - acc: 0.0520
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.1738 - acc: 0.0360
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.0892 - acc: 0.0080
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.0081 - acc: 0.0060
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.9236 - acc: 0.0080
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.8228 - acc: 0.0080
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.7323 - acc: 0.0120
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.6481 - acc: 0.0180
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.5760 - acc: 0.0350


In [39]:
#入力ディクショナリを使った適合
model.fit({'text':text, 'question': question}, answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.4972 - acc: 0.0340
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.4275 - acc: 0.0530
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.3537 - acc: 0.0550
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.2828 - acc: 0.0640
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.2216 - acc: 0.0700
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.1587 - acc: 0.0840
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.0978 - acc: 0.0890
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 5.0350 - acc: 0.1060
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 4.9893 - acc: 0.1040
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/step - loss: 4.9137 - acc: 0.1200


<h2>7.1.3 多出力モデル

In [40]:
from keras import layers
from keras import Input
from keras.models import Model

In [41]:
vocabulary_size = 50000
num_income_groups = 10

In [42]:
posts_input = Input(shape=(None,), dtype='int32', name='posts')

In [43]:
#本にタイポあり
#embedded_posts = layers.Embedding(256, vocabulary_size)(post)
embedded_posts = layers.Embedding( vocabulary_size, 256)(posts_input)

In [45]:
#出力フィルタ（つまりカーネルサイズの枠で切り取った部分の奥行き）=128
#カーネルの枠のsize = 5
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)

In [46]:
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
#出力空間次元数=128
x = layers.Dense(128, activation='relu' )(x) 

In [47]:
#出力層に名前をつける
age_prediction = layers.Dense(1, name='age')(x)

In [48]:
income_prediction = layers.Dense(num_income_groups,
                                activation='softmax',
                                name = 'income')(x)

In [49]:
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

In [50]:
model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

<h3>複数の損失を定義するコンパイル

In [51]:
# model.compile(optimizer='rmsprop',
#              loss=['mse',
#                   'categorical_crossentropy',
#                   'binary_crossentropy'])

#出力層に名前をつけている場合
model.compile(optimizer='rmsprop',
             loss={'age':'mse',
                  'income':'categorical_crossentropy',
                  'gender':'binary_crossentropy'})